In [1]:
from dotenv import load_dotenv
from langchain_classic.output_parsers import ResponseSchema


load_dotenv()

from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# ChatPromptTemplate
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# basic_qa_chain = model | StrOutputParser()

# response = basic_qa_chain.invoke("请自我介绍。")
# print(response)

# 解析布尔输出
from langchain_classic.output_parsers.boolean import BooleanOutputParser
# BooleanOutputParser

prompt_template = ChatPromptTemplate([
    ("system", "你是一个乐于助人的帮手，请根据用户的问题给出回答。"),
    ("user", "这是用户的问题：{topic}")
])

math_qa_chain = prompt_template | model | StrOutputParser()

question1 = \
"""
Alice and Bob are each holding some integer number of sweets. Alice says to Bob: “If
we each added the number of sweets we’re holding to our (positive integer) age, my answer would
be double yours. If we took the product, then my answer would be four times yours.” Bob replies:
“Why don’t you give me five of your sweets because then both our sum and product would be equal.”
What is the product of Alice and Bob’s ages?

"""

question = \
"""
what is 1 / 0?
"""

result = math_qa_chain.invoke({"topic": question})

print(result)


ModuleNotFoundError: No module named 'langchain_rag_demo'

In [3]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

schemas = [
    ResponseSchema(name="name", description="用户的姓名"),
    ResponseSchema(name="age", description="用户的年龄")
]

parser = StructuredOutputParser.from_response_schemas(schemas)

prompt = PromptTemplate.from_template(
    "请根据以下内容提取用户信息，并返回 JSON 格式：\n{input}\n\n{format_instructions}"
)

format_instructions = parser.get_format_instructions()
print(format_instructions)

chain = (
    prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

result = chain.invoke({"input": "用户叫李雷，今年25岁，是一名工程师。"})

print(result)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // 用户的姓名
	"age": string  // 用户的年龄
}
```
{'name': '李雷', 'age': '25'}


LECL -- LangChainExpressionLanguage
函数式管道风格，允许通过管道符连接两个元素，实现功能的串联
invoke
stream
batch

In [1]:
from dotenv import load_dotenv
load_dotenv()
# 构建复合链
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser

from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

news_gen_prompt = PromptTemplate.from_template(
    "请根据以下新闻标题撰写一则简短的新闻内容，100字以内：\n\n{title}"
)

# model输出一个AIMessage
news_chain = news_gen_prompt | model

schemas = [
    ResponseSchema(name="time", description="事件发生的时间"),
    ResponseSchema(name="location", description="事件发生的地点"),
    ResponseSchema(name="event", description="发生的具体事件")
]

parser = StructuredOutputParser.from_response_schemas(schemas)

summary_prompt = PromptTemplate.from_template(
    "请从下面的新闻内容中提取关键信息，并返回结构化的json格式：\n\n{news}\n\n{format_instructions}"
)

summary_chain = (
    summary_prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

combined_chain = news_chain | summary_chain

response = combined_chain.invoke({"title": "陨石撞击地球"})

print(response)


{'time': '昨日凌晨', 'location': '太平洋无人海域', 'event': '一颗直径约10米的陨石坠入，未造成人员伤亡及财产损失，相关机构正对陨石碎片展开研究'}


In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

from langchain_core.output_parsers import StrOutputParser

model = init_chat_model("deepseek-chat", model_provider="deepseek")
chatbot_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="你叫小智，是一名乐于助人的助手。"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
basic_qa_chain = chatbot_prompt | model | StrOutputParser()
messages_list = [
    HumanMessage(content="你好，我叫程明，好久不见。"),
    AIMessage(content="你好呀！我叫小智，是一名乐于助人的ai助手，很高兴认识你。")
]

question = "你好，请详细介绍一下你自己。"
messages_list.append(HumanMessage(content=question))
# result = basic_qa_chain.invoke({"messages": messages_list})

async for chunk in basic_qa_chain.astream({"messages": messages_list}):
    print(chunk, end="", flush=True)

# print(result)

你好！我是小智，一个由深度求索公司开发的AI助手。很高兴能为你详细介绍我自己：

**我的能力：**
- 知识问答：涵盖科学、历史、文化、生活等各个领域
- 文本处理：写作、翻译、总结、改写等
- 逻辑推理：数学计算、问题分析、方案建议
- 创意生成：故事创作、诗歌写作、头脑风暴
- 编程帮助：代码解释、调试建议、算法思路

**我的特点：**
- 完全免费使用
- 支持128K长上下文对话
- 会主动承认知识盲区
- 注重回答的准确性和安全性

**我的限制：**
- 知识截止到2024年7月
- 无法处理实时信息
- 没有个人情感和主观体验

有什么具体想了解的吗？或者想试试我的什么功能？😊

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### Langchain 接入工具

- 基本流程

    - MCP
    - LangChain 内置工具


In [5]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool # Python交互式解释环境，能够保存运行上下文


# 电信用户流失预测
path = "/mnt/d/aistudy/dataset/langchain/telco/Telco_customer_churn.xlsx"
df = pd.read_excel(path)
# df.head(5)
tool = PythonAstREPLTool(locals={"df": df})
# tool.invoke("df.columns")

from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
model = init_chat_model("deepseek-chat", model_provider="deepseek")

llm_with_tool = model.bind_tools([tool])

# response = llm_with_tool.invoke(
#     "我有一张表名为df，请告诉我表的列名"
# )

from langchain_core.output_parsers import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(
    key_name=tool.name,
    first_tool_only=True
)
chain = llm_with_tool | parser
# response = chain.invoke("我有一张表名为df，请告诉我表的列名")

system_prompt = \
"""
你可以访问一个名为'df'的pandas数据框，你可以使用df.head().to_markdown()查看数据集的基本信息。
请根据用户提出的问题，编写Python代码来混搭，只返回代码，不返回其他内容。只允许使用pandas和内置库。
"""
from langchain_core.messages import SystemMessage, HumanMessage
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{question}")
    ]
)

from langchain_core.runnables import RunnableLambda
def _extract_code(message):
    return message["query"]

extract_code = RunnableLambda(_extract_code)
chain = prompt | llm_with_tool | parser | extract_code | tool
# chain = prompt
response = chain.invoke({"question": "统计已流失的用户有多少人？"})
response

ChatPromptValue(messages=[SystemMessage(content="\n你可以访问一个名为'df'的pandas数据框，你可以使用df.head().to_markdown()查看数据集的基本信息。\n请根据用户提出的问题，编写Python代码来混搭，只返回代码，不返回其他内容。只允许使用pandas和内置库。\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='统计已流失的用户有多少人？', additional_kwargs={}, response_metadata={})])

In [22]:
from langchain_core.tools import tool
import requests
from dotenv import load_dotenv
load_dotenv(override=True)
import os
# https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}

@tool
def get_weather(city):
    """
    获取指定城市当前的天气情况
    :param city: 必要参数，城市名，应当为英文形式
    :return: 接口响应，json格式
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": os.getenv("OWM_APPID"),
        "units": "metric",
        "lang": "zh_cn"
    }
    response = requests.get(url, params=params)
    return response.json()

print(get_weather.name)
print(get_weather.description)
print(get_weather.args)


from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv(override=True)
chat_model = init_chat_model("deepseek-chat", model_provider="deepseek")
chat_model = chat_model.bind_tools([get_weather])

from langchain_core.output_parsers import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name=get_weather.name, first_tool_only=True)

chain = chat_model # | parser # | get_weather

# response = chain.invoke("请问北京天气怎么样？")

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_prompt = PromptTemplate.from_template(
"""
你将收到一段 JSON 格式的天气数据，请用简洁自然的方式将其转述给用户。
以下是天气 JSON 数据：

```json
{weather_json}
```

请将其转换为中文天气描述，例如：
“北京当前天气晴，气温为23℃，湿度58%，风速2.1米/秒。”
只返回一句话描述，不要其他说明或解释。
"""
)

output_chain = output_prompt | chat_model | StrOutputParser()

# weather_json = chain.invoke("请问成都天气怎么样？")
#
# response = output_chain.invoke({"weather_json": weather_json})

from langchain_core.runnables import RunnableLambda
combined_chain = chain # | RunnableLambda(lambda x: {"weather_json": x}) | output_chain
response = combined_chain.invoke("请问华盛顿的天气怎么样？")
print(response)

get_weather
获取指定城市当前的天气情况
:param city: 必要参数，城市名，应当为英文形式
:return: 接口响应，json格式
{'city': {'title': 'City'}}
content='我需要确认一下您指的是哪个华盛顿。美国有两个主要的地方都叫华盛顿：\n\n1. **华盛顿州**（Washington State）- 美国西北部的一个州\n2. **华盛顿特区**（Washington D.C.）- 美国的首都\n\n您想知道的是哪个地方的天气呢？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 325, 'total_tokens': 382, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 5}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '0fbea8ab-831d-4e36-bed8-72bc98c3dabc', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bcaf0-7b9b-7071-8d6f-50bcffb7a88e-0' usage_metadata={'input_tokens': 325, 'output_tokens': 57, 'total_tokens': 382, 'input_token_details': {'cache_read': 320}, 'output_token_details': {}}


In [2]:
from langchain_classic.agents import create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv()
import os
import requests

@tool
def get_weather(city):
    """
    获取指定城市当前的天气情况
    :param city: 必要参数，城市名，应当为英文形式
    :return: 接口响应，json格式
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": os.getenv("OWM_APPID"),
        "units": "metric",
        "lang": "zh_cn"
    }
    response = requests.get(url, params=params)
    return response.json()

@tool
def write_file(content):
    """
    将输入内容写入本地文件
    :param content: 待写入内容
    :return: 写入提示
    """
    return "文件写入成功"

tools = [get_weather, write_file]

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息，如果用户需要将查询结果写入文件，请使用write_file工具。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


from langchain.chat_models import init_chat_model
model = init_chat_model("deepseek-chat", model_provider="deepseek")

agent = create_tool_calling_agent(model, tools, prompt)

from langchain_classic.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "请问北京和杭州今天的天气怎么样？哪个地方更热？将结果写入文件。"})
print(response)




> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Beijing'}`
responded: 我来帮您查询北京和杭州的天气情况，然后比较哪个地方更热，并将结果写入文件。

首先，让我查询这两个城市的天气：

{'coord': {'lon': 116.3972, 'lat': 39.9075}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': '多云', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': -4.06, 'feels_like': -4.06, 'temp_min': -4.06, 'temp_max': -4.06, 'pressure': 1030, 'humidity': 51, 'sea_level': 1030, 'grnd_level': 1024}, 'visibility': 10000, 'wind': {'speed': 1.31, 'deg': 352, 'gust': 1.93}, 'clouds': {'all': 61}, 'dt': 1768713000, 'sys': {'type': 1, 'id': 9609, 'country': 'CN', 'sunrise': 1768692778, 'sunset': 1768727756}, 'timezone': 28800, 'id': 1816670, 'name': 'Beijing', 'cod': 200}
Invoking: `get_weather` with `{'city': 'Hangzhou'}`


{'coord': {'lon': 120.1614, 'lat': 30.2937}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': '晴，少云', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 14.95, 'feels_like': 13.86, 'temp_min': 14.95, '

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

# Tavily Search
from langchain_tavily import TavilySearch
search = TavilySearch(max_results=2)
# search.invoke("苹果2025WWDC发布会")

from langchain.chat_models import init_chat_model
model = init_chat_model("deepseek-chat", model_provider="deepseek")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一名助人为乐的助手，可以使用工具进行网络搜索，获取实时信息。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

tools = [search]

from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
agent = create_tool_calling_agent(model, tools, prompt)

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

executor.invoke({"input": "请问苹果的2025WWDC发布会的召开时间是？"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '苹果2025年WWDC发布会召开时间 日期', 'search_depth': 'advanced', 'time_range': 'month'}`
responded: 我来帮您搜索苹果2025年WWDC发布会的具体召开时间。

{'query': '苹果2025年WWDC发布会召开时间 日期', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.instagram.com/p/DS9uBD9kZQH/', 'title': '2025年對蘋果而言是豐收的一年，在iPhone強勁銷售帶動 ...', 'content': '至於iOS 19系統則預計在6月10日的WWDC大會上首度亮相，開發者測試版會在會後立即推出，正式版則安排在9月與新iPhone同步發布。 #WavingCatHK #iOS #iPhone #AAPL', 'score': 0.7474137, 'raw_content': None}, {'url': 'https://www.pconline.com.cn/focus/2039/20395192.html', 'title': 'iPhone 路线图全曝光：两年七款新机，阔折叠、四曲面全来了！', 'content': '���ϳ�OPPO������˫2�����ط������»���ΪFind X9s \n\n  2025��12��16����Ϣ��OPPO������˫2������Ӱ�\U00077f70�����Ӧ����Find X9s�»����÷�����������200MP�����������˸��������ɱ����ƣ���ģ�������\u0379����ϸ������ơ��˴β�����ʾOPPO��Ӱ�������ϵ�ǰհ̽�����������Ƿ�������δȷ�ϡ��˾ٷ�ӳ��Ʒ�ƶԸ߶�Ӱ���콢�г��ĳ��������봴��\u05f7����\n\n  ʮ�� 2025-12-16 15:

{'input': '请问苹果的2025WWDC发布会的召开时间是？',
 'output': '太好了！我从苹果开发者官网找到了官方信息。根据搜索结果，苹果已经正式宣布了WWDC25的召开时间：\n\n**苹果2025年WWDC发布会时间：**\n\n**📅 日期：2025年6月9日 - 6月13日**（周一至周五）\n\n**🎯 关键信息：**\n- **主题演讲（Keynote）**：将于**6月9日（周一）**举行\n- **平台状态更新（Platforms State of the Union）**：也将在6月9日进行\n- **活动形式**：在线举行，全球开发者可以线上参与\n- **地点**：苹果总部Apple Park（库比蒂诺）\n\n**📱 预计发布内容：**\n根据传统，WWDC主要关注软件更新，预计将发布：\n- iOS 19（或iOS 26，取决于苹果的命名方式）\n- iPadOS 19/26\n- macOS 15/26\n- watchOS 11/26\n- tvOS 18/26\n- 以及可能的硬件产品更新\n\n**💡 重要提示：**\n苹果通常在每年3月左右正式宣布WWDC的具体日期，然后在6月初举行活动。今年的WWDC25已经确定在6月9-13日举行，开发者可以提前做好准备。\n\n您是想了解如何观看直播，还是对WWDC的具体内容有更多兴趣呢？'}

In [2]:
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

search = search_database
print(search.name)
print(search.description)
print(search.args)

search_database
Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
{'query': {'title': 'Query', 'type': 'string'}, 'limit': {'default': 10, 'title': 'Limit', 'type': 'integer'}}


In [1]:
# PlayWrite
! playwright install

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
(node:55744) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [                    ] 0% 0.0s164.7 MiB [                    ] 0% 259.0s164.7 MiB [                    ] 0% 752.2s164.7 MiB [                    ] 0% 602.4s164.7 MiB [                    ] 0% 566.9s164.7 MiB [                    ] 0% 516.6s164.7 MiB [                    ] 0% 496.5s164.7 MiB [                    ] 0% 467.3s164.7 MiB [                    ] 0% 440.8s164.7 MiB [                    ] 0% 585.7s164.7 MiB [                    ] 0% 479.1s164.7 MiB [                    ] 0% 412.1s164.7 MiB [                    ] 0% 365.7s164.7 MiB [                    ] 0% 353.0s164.7 MiB [                    ] 0% 328

In [2]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_sync_playwright_browser
from langchain_classic import hub
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv
load_dotenv()

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")

sync_browser = create_sync_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(sync_browser=sync_browser)
tools = toolkit.get_tools()

prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt)

model = init_chat_model("deepseek-chat", model_provider="deepseek")

agent = create_openai_tools_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

command = {
    "input": "访问这个网站https://www.zhihu.com/hot，简要介绍前3条热点信息。"
}

response = agent_executor.invoke(command)

print(response)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.